<a href="https://colab.research.google.com/github/denistoo749/Academic-Success-Classification/blob/main/academic_success_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# # Unzip the file
# !unzip '/content/drive/MyDrive/Academic Success Classification/playground-series-s4e6.zip' -d '/content/drive/MyDrive/Academic Success Classification/data/'

Archive:  /content/drive/MyDrive/Academic Success Classification/playground-series-s4e6.zip
  inflating: /content/drive/MyDrive/Academic Success Classification/data/sample_submission.csv  
  inflating: /content/drive/MyDrive/Academic Success Classification/data/test.csv  
  inflating: /content/drive/MyDrive/Academic Success Classification/data/train.csv  


Import necessary tools

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [3]:
# Read the train dataset
df = pd.read_csv('/content/drive/MyDrive/Academic Success Classification/data/train.csv')
df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76518 entries, 0 to 76517
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              76518 non-null  int64  
 1   Marital status                                  76518 non-null  int64  
 2   Application mode                                76518 non-null  int64  
 3   Application order                               76518 non-null  int64  
 4   Course                                          76518 non-null  int64  
 5   Daytime/evening attendance                      76518 non-null  int64  
 6   Previous qualification                          76518 non-null  int64  
 7   Previous qualification (grade)                  76518 non-null  float64
 8   Nacionality                                     76518 non-null  int64  
 9   Mother's qualification                 

In [6]:
df.isna().sum()

id                                                0
Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance                        0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                                            0
Scholarship 

In [7]:
# Create X and y
X = df.drop('Target', axis=1)
y = df['Target']

In [16]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

def train_and_evaluate_models(X, y):
    # Example data split - adjust according to your actual data split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 1. Encode the Target Variable
    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    # 2. Encode Categorical Features in X
    categorical_features = [
        'Marital status', 'Application mode', 'Daytime/evening attendance',
        'Previous qualification', 'Nacionality', 'Mother\'s qualification',
        'Father\'s qualification', 'Mother\'s occupation', 'Father\'s occupation',
        'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date',
        'Gender', 'Scholarship holder', 'International'
    ]

    # Convert categorical features to dummy variables
    X_train_encoded = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
    X_test_encoded = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

    # Ensure that the encoded training and test sets have the same columns
    X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

    # 3. Create a pipeline for Logistic Regression that scales the data and applies logistic regression
    log_reg_pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

    # Train and Evaluate the Logistic Regression Model
    log_reg_pipeline.fit(X_train_encoded, y_train_encoded)
    log_reg_score = log_reg_pipeline.score(X_test_encoded, y_test_encoded)
    print(f"LogisticRegression score after encoding and scaling: {log_reg_score}")

    # 4. Train and Evaluate the Random Forest Classifier
    rf_classifier = RandomForestClassifier()

    # Train the model
    rf_classifier.fit(X_train_encoded, y_train_encoded)
    rf_score = rf_classifier.score(X_test_encoded, y_test_encoded)
    print(f"RandomForestClassifier score: {rf_score}")

In [15]:
train_and_evaluate_models(X, y)

LogisticRegression score after encoding and scaling: 0.8218112911657083
RandomForestClassifier score: 0.8246210141139572


# Hyperparameter Tuning using RandomizedSearchCV